In [3]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.2f}'.format

In [5]:
# Build base url
HOST = 'https://api.census.gov/data'
dataset = 'acs/acs1'

# Specify Census variables and other predicates
get_vars = ['NAME', 'B01003_001E', 'B03001_001E', 'B03001_007E']
predicates = {}
predicates['get'] = ",".join(get_vars)
pred